## 해외축구 경기 정보 받기
- 관심있는 축구 경기 정보 받기
- 보고싶은 경기를 시청할 수 없을 때, 스코어에 변화가 생길떄마다 현재 경기 스코어와 득점 정보를 전송
- 보고싶은 경기가 진행되는 카테고리(컵, 리그)를 먼저 정해야 함
- (경기 시작 전) 라인업 정보가 뜨면 각 팀의 선발 명단 크롤링
- (경기 종료 후) 최종 스코어와 세부 기록 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup as bs

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [2]:
schedule_url = 'https://sports.news.naver.com/wfootball/schedule/index'

category = ['epl', 'primera', 'bundesliga', 'seria', 'ligue1',
            'denmark', 'champs', 'europa', 'uecl', 'facup',
            'carlingcup', 'copadelrey', 'clubworldcup']

lineup_url = 'https://m.sports.naver.com/game/2023081465041892243/lineup'
record_url = 'https://m.sports.naver.com/game/2023081465041892243/record'